In [1]:
import pandas as pd

In [2]:
import requests, os, re
import time, random
from datetime import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options as c_op
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#1#prima cosa da fare: definire il web driver (con torr dentro), ma ora non lo usiamo
def webDriver(driver_type: str = "chrome", headless: bool=False, 
              driver_path: str = None,
              tor: bool = True,
              torrc: str = "/etc/tor/torrc"):
    
    if driver_type == "chrome":
        options = c_op()
        if headless:
            options.add_argument("--headless")
        
#2# start chrome driver instance with TOR
        web = webdriver.Chrome(options=options, executable_path=driver_path)

    elif driver_path == "firefox":
        options = Options()
        options.headless = headless
        
        if torrc is not None and tor is True:

            options.set_preference("network.proxy.type", 1)
            options.set_preference("network.proxy.socks_version", 5)
            options.set_preference("network.proxy.socks", "127.0.0.1")
            options.set_preference("network.proxy.socks_port", 9050)
            options.set_preference("network.proxy.socks_remote_dns", False)
        
        options.set_preference("network.proxy.type", 1)
        options.set_preference("dom.webdriver.enabled", False)
        options.set_preference("useAutomationExtension", False)
        options.set_preference("devtools.jsonview.enabled", False)

        
        web = webdriver.Firefox(
            options=options,
            executable_path=driver_path,
        )
        
        web.get("https://check.torproject.org/")
    
    return web

In [3]:
def get_source(webdriver):
    
    source = bs(webdriver.page_source, "lxml")
    return source

In [4]:
data=pd.read_excel('yourdata')

In [5]:
data

,AccountNo,Org_cleaned
0,44,3I GROUP
1,53,ABERTIS INFRAESTRUCTURAS
2,78,ABERDEEN ASSET MANAGEMENT
3,87,A2A
4,97,AAREAL BANK
...,...,...
845,74136,GALENICA
846,74137,METRO
847,831040,ATLANTICA YIELD
848,832133,SENSIRION HOLDING


In [6]:
if __name__ == "__main__":
    client = MongoClient()
    db = client["your_admin"]
    coll = db["your_db"]

    webdriver = webDriver(driver_type="chrome", headless=False, driver_path="/home/grazia/Documents/chromedriver")
    webdriver.get('https://duckduckgo.com/')

    queries = data["your_col"].unique().tolist()
    total = []

    for q in queries:
        d = {"name": q, "did_u_mean": "", "url": "", "text": ""}
        search_box = WebDriverWait(webdriver, 20).until(EC.element_to_be_clickable((By.NAME, "q")))
        search_box.send_keys(q)
        search_box.submit()

        time.sleep(1)

        source = get_source(webdriver)
        did_u_mean = source.find("div", {"data-testid": "spelling-message"})
        if did_u_mean:
            links = did_u_mean.find_all("a")
            if links:
                link = links[0]
                spell = link.text.strip()
                d["did_u_mean"] = spell

        results_list = source.find("ol", class_="react-results--main")

        
        if results_list:
            # Get the first result
            first_result = results_list.find("li", {"data-layout": "organic"})
            if first_result:
                # Extract the URL
                url_tag = first_result.find("a", {"data-testid": "result-title-a"})
                if url_tag:
                    url = url_tag.get("href")

                # Extract the snippet
                snippet_tag = first_result.find("div", {"data-result": "snippet"})
                if snippet_tag:
                    snippet = snippet_tag.text.strip()

                # Add the URL and snippet to the dictionary d
                d["url"] = url
                d["text"] = snippet

        

        total.append(d)
        check = coll.find_one({"name": d.get("name")})
        if not check:
            coll.insert_one(d).inserted_id
            print(f"Add name {q}, info: {d.get('did_u_mean')}")
        else:
            print(f"Not added {q}, {d}")
            pass

        webdriver.find_element(By.XPATH, "//input[@name='q']").clear()  # syntax for selenium search, to clear the line


/tmp/ipykernel_1036649/878236919.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  web = webdriver.Chrome(options=options, executable_path=driver_path)


Add name 3I GROUP, info: 
Add name ABERTIS INFRAESTRUCTURAS, info: 
Add name ABERDEEN ASSET MANAGEMENT, info: 
Add name A2A, info: 
Add name AAREAL BANK, info: 
Add name ABN AMRO HOLDING, info: 
Add name ACCIONA, info: 
Add name ACCORHOTELS, info: 
Add name ACERINOX, info: 
Add name ADP (AEROPORTS DE PARIS), info: 
Add name ADMIRAL GROUP, info: 
Add name ACTELION, info: 
Add name ADECCO GROUP, info: 
Add name ADVA OPTICAL NETWORKING SE, info: 
Add name AEGON, info: 
Add name DENTSU AEGIS NETWORK, info: 
Add name ABB, info: 
Add name ACEA, info: 
Add name ACS ACTIVIDADES DE CONSTRUCCION Y SERVICIOS, info: 
Add name AIXTRON SE, info: 
Add name AIR FRANCE  KLM, info: 
Add name AIR LIQUIDE, info: 
Add name ALCATEL  LUCENT, info: 
Add name AKER ASA, info: 
Add name AKTIA BANK, info: 
Add name ALFA LAVAL CORPORATE, info: 
Add name ALMA MEDIA CORPORATION, info: 
Add name ALPHA BANK, info: 
Add name ALLIANCE TRUST, info: 
Add name ALLIANZ SE, info: 
Add name AIB GROUP, info: 
Add name ALSTOM, 